# Parabolic Partial Differential Equations

## Diffusion Equation

Here we solve the diffusion equation
$$
\frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial x^2} \quad ,
$$
with $D=1$, using as an initial condition a gaussian profile $u(x,t=0)=\exp{[-(x-x_0)^2]}$ with $x_0=5$.

We use a domain $x\in [0,10]$ with periodic boundary conditions.

We solve this by using the FTCS methods which is stable for the diffusion equation as long as $\Delta t \leq \frac{\Delta x^2}{(2D)}$.

In [ ]:
import numpy as np

#define the diffusion coefficient D
D=1.0

# Define the domain
L = 10.0     # Domain length
nx = 101    # Number of grid points
dx = L/(nx-1)   # Grid spacing
x = np.linspace(0, L, nx)

# Define the time step and the final time
cf = 0.5
dt = cf*dx**2/(2*D)   #Time step
t_final = 5.0

print('nx=',nx)
print('dx=',dx)
print('dt=',dt)
print('Number of iterations=',t_final/dt)


# Define the initial condition
x0=5
u_initial = np.exp(-(x-x0)**2)

# Initialize the solution array
u_current = u_initial.copy()

#we create arrays where we will store the time and the l2norm
l2norm=[]
l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))

time=[]
time.append(0.0)

In [ ]:
import matplotlib.pyplot as plt
import os
##create directory where to save images
print(os.getcwd())

os.makedirs('./images_parabolic')

os.listdir('./')

In [ ]:
# Initilize time and iteration counter
t = 0.0
i = 0

#save the initial conditions
plt.plot(x, u_current)
plt.title('Time='+str(round(t,2)))
plt.ylim(0,1.1)
plt.savefig('./images_parabolic/fig_'+str(i).zfill(5)+'.png', dpi=200)
plt.close()

#solve the diffusion equation
while t < t_final:
    # Compute the new solution using the FTCS method
    # Note: np.roll(u_current, -1) is equivalent to u(j+1) and
    #       np.roll(u_current,  1) is equivalent to u(j-1)
    # using np.roll is equivalent to use periodic boundary conditions
    u_next = u_current + D*dt/(dx**2)*(np.roll(u_current, 1) - 2*u_current + np.roll(u_current, -1))    
    
    # Update the solution
    u_current = u_next.copy()
    
    
    #advance the time 
    t += dt
    i += 1
    
    #compute the l2 norm and add the time to the time vector
    l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
    time.append(t)
    
    #plot the current result and save in an image every 10 iterations
    if (i%10==0):
        plt.plot(x, u_current)
        plt.title('Time='+str(round(t,2)))
        plt.ylim(0,1.1)
        plt.savefig('./images_parabolic/fig_'+str(i).zfill(5)+'.png', dpi=200)
        plt.close()


In [ ]:
# Plot the final solution
plt.plot(x, u_initial, label='Initial')
plt.plot(x, u_current, label='Final')
plt.title('Time='+str(round(t,2)))
plt.ylim(0,1.1)
plt.legend()
plt.show()

In [ ]:
# set the directory where your images are stored
directory = "./images_parabolic/"

# get the list of image files in the directory
files = os.listdir(directory)

print(files, '\n')

# sort the files in alphanumeric order
files=sorted(files)

print(files)




In [ ]:
import imageio
with imageio.get_writer('./movie.mp4', mode='I', quality=10) as writer:
    for file in files:
        image = imageio.imread('./images_parabolic/'+file)
        writer.append_data(image)
        
files=[]


In [ ]:
# don't worry about the code in this cell, it is just to let you 
# display the movies you generated above in Jupyter notebook
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./movie.mp4" type="video/mp4">
</video></div>""")

In [ ]:
plt.plot(time,l2norm)
plt.show()

### Exercise

Try changing the resolution (use at least three different resolutions) and compare the final solutions.

Try also changing the initial data using periodic functions. For example:

1. $u(x,t=0) = \sin(2\pi x /L) $
2. $u(x,t=0) = \sin(8\pi x /L) $
3. $u(x,t=0) = \sin(2\pi x /L) + \sin(8\pi x /L) $

where $L=10$ as before. Compare the results in the different cases and comment.

## Alternative Numerical Methods

The FTCS method is stable for the diffusion equation, but it requires $\Delta t \leq \Delta x^2/(2D)$. This means that if I increase the number of points in x by a factor of 2, the number of iterations in time increase by a factor of 4. FTCS is therefore computationally expensive.

There are alternative methods that can be used to avoid this issue.

### BTCS Method

The Backward in Time - Centered in Space (BTCS) is an implict method that is stable for any choice of $\Delta t$. The method is still first order in time, so the error will go like $\Delta t$. Hence it is always a good idea to choose a sufficiently small $\Delta t$, but it can be much larger than the one to be used in the FTCS method.

$$
u^{n+1}_j = u^n_j + D \frac{\Delta t}{\Delta x^2} \left( u^{n+1}_{j+1} - 2 u^{n+1}_j + u^{n+1}_{j-1} \right)
$$

### Crank-Nicholson Method

This method computes $\frac{\partial u}{\partial t}$ at time $n+1/2$, by using a centered derivative with half step $\Delta t/2$, and it computes the derivative in space at $n+1/2$ by taking an average of the space derivative computed at $n$ and $n+1$:

$$
u^{n+1}_j = u^n_j + D \frac{\Delta t}{2\Delta x^2} \left[ \left( u^{n+1}_{j+1} - 2 u^{n+1}_j + u^{n+1}_{j-1} \right) + \left( u^{n}_{j+1} - 2 u^{n}_j + u^{n}_{j-1} \right)\right]
$$

This method is also unconditionally stable as the BTCS, but it is second order both in time and in space.

### Dufort-Frankel Method

This method uses a second order approximation in time (like in the Leapfrog scheme) and a second order approximation in space, as the FTCS, but replacing $u^n_j$ with the average $\frac{1}{2}(u^{n+1}_j+u^{n-1}_j)$:

$$
\frac{u^{n+1}_j-u^{n-1}_j}{2\Delta t} = \frac{D}{\Delta x^2}\left[ u^n_{j+1} -\frac{2}{2} \left( u^{n+1}_j + u^{n-1}_j \right) + u^n_{j-1} \right]
$$

which is equvalent to (just rearrange the terms in the expression above)

$$
u^{n+1}_j = \left( \frac{1-2\alpha}{1+2\alpha} \right) u^{n-1}_j + \left( \frac{2\alpha}{1+2\alpha} \right) (u^n_{j+1}+u^n_{j-1})
$$

where $\alpha = D \Delta t/\Delta x^2 $.

This method is explicit, so easier to implement, it is unconditionally stable, and it is second order both in space and in time. Like the Leapfrog scheme it has the disadvantage of requiring 3 timelevels instead of 2.

### Exercise

Use one of the above methods to solve the diffusion equation using one of the initial data you used before. Comment on the results, focusing on the resolution required to get a result similar to the one obtained with the FTCS method.

Note: the implicit methods will require you to solve an algebraic system of equations and work with a banded matrix (you can use python [scipy](https://docs.scipy.org/doc/scipy/index.html) to solve it, e.g., [solve_banded](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_banded.html)). The Dufort-Frankel method instead will require two time levels to start. You can do a first step with FTCS and then continue with the Dufort-Frankel method.